In [1]:
import requests
import pandas as pd
import json
import requests

import os.path

In [2]:
rerun = False

# Get Player IDs

In [3]:
if rerun and os.path.isfile('links_for_player_ids.json') == False:
    ref_list = []
    for num in range(1, 7):
        url = 'https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes?limit=1000&active=true&page={}'.format(num)
    
        r = requests.get(url)
        d = r.json()
    
        ref_list += d['items']
    with open('links_for_player_ids.json', 'w') as f:
        json.dump(ref_list, f, ensure_ascii=False, indent=4)

In [4]:
player_id_data = json.load(open('links_for_player_ids.json'))

# Get Specific Player Info/QBRs for NFL

In [5]:
player_ids = []
for player in range(0, len(player_id_data)):
    player_ids.append(player_id_data[player]['$ref'][72:int(player_id_data[player]['$ref'].find('?'))])

In [9]:
full_dict = {}

for player_id in player_ids:

    # This gets every single players info (height, weight, team, etc.)
    info_url = "http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes/{}?lang=en&region=us".format(player_id)
    
    player_info_req = requests.get(info_url)
    player_info = player_info_req.json()

    # Filters for QBs only
    if player_info['position']['id'] == '8':

        # This gets all the stats for every season a QB played in the NFL
        # Note: If no stats this pops an error
        seasons_url = 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes/{}/statisticslog?lang=en&region=us'.format(player_id)

        player_seasons_req = requests.get(seasons_url)
        player_seasons = player_seasons_req.json()

        # Iterates through every link to each season of stats
        if 'entries' in player_seasons.keys():
            print(player_id, player_info['displayName'])
            if 'country' in player_info['birthPlace'].keys():
                country = player_info['birthPlace']['country']
            else:
                country = 'Not Given'
                
            full_dict[player_id] = {'player_name':player_info['displayName'],
                                    'date_of_birth':player_info['dateOfBirth'],
                                    'city':player_info['birthPlace']['city'],
                                    'state':player_info['birthPlace']['state'],
                                    'country':country,
                                    'height':player_info['height'],
                                    'weight':player_info['weight']
                                   }

            # This is done in case the player wasn't drafted or is 
            # a rookie. If either are true, this pops an error
            if 'debutYear' in player_info.keys():
                full_dict[player_id]['debut_year'] = player_info['debutYear']
            elif 'debutYear' not in player_info.keys() and player_info['experience']['years'] == 0:
                full_dict[player_id]['debut_year'] = 2023
            elif 'debutYear' not in player_info.keys() and player_info['experience']['years'] != 0:
                full_dict[player_id]['debut_year'] = player_seasons['entries'][-1]['season']['$ref'][71:75]
            else:
                raise Exception("Player is not an undrafted free agent or in debut year") 
                
            
            # This is done for Undrafted Free Agents. It pops an error if
            # they weren't drafted
            if 'draft' in player_info.keys():
                full_dict[player_id]['draft'] = {
                                        'year':player_info['draft']['year'],
                                        'round':player_info['draft']['round'],
                                        'selection':player_info['draft']['selection']
                                    }
            else:
                full_dict[player_id]['draft'] = 'UDFA'

            for stats in player_seasons['entries']:
                nfl_stats = {}
                player_stats_req = requests.get(stats['statistics'][0]['statistics']['$ref'])
                player_stats = player_stats_req.json()
    
                # Gets all types of NFL QBRs for each season
                for qbr in player_stats['splits']['categories'][1]['stats']:
                    if ("QBR" in qbr['name']) or ("quarterbackRating" in qbr['name']):
                        year = stats['statistics'][0]['statistics']['$ref'][71:75]
                        if year in nfl_stats.keys():
                            nfl_stats[year].update({qbr['name']:qbr['value']})
                        else:
                            nfl_stats[year] = {qbr['name']:qbr['value']}
                            
                if 'nfl' not in full_dict[player_id].keys():
                    full_dict[player_id]['nfl'] = nfl_stats
                else:
                    full_dict[player_id]['nfl'].update(nfl_stats)
    else:
        pass

2574511 Brandon Allen
3918298 Josh Allen
3115293 Kyle Allen
4434153 Tyson Bagent
15948 Matt Barkley
2979520 C.J. Beathard
3040535 Kurt Benkert
3116188 David Blough
4046678 Ian Book
16724 Blake Bortles
3045169 Tim Boyle
16252 Tyler Bray
2578570 Jacoby Brissett
4035312 Anthony Brown
3886812 Jake Browning
3915511 Joe Burrow
16757 Derek Carr
4259592 Sean Clifford
2576261 Connor Cook
14880 Kirk Cousins
4240069 Malik Cunningham
14012 Andy Dalton
12471 Chase Daniel
3912547 Sam Darnold
4240391 Tommy DeVito
3895785 Ben DiNucci
3044720 Joshua Dobbs
2574630 Jeff Driskel
4035003 Jacob Eason
4241820 Sam Ehlinger
16821 David Fales
4362887 Justin Fields
11252 Joe Flacco
14877 Nick Foles
4240689 Jake Fromm
13987 Blaine Gabbert
16760 Jimmy Garoppolo
16809 Garrett Gilbert
15837 Mike Glennon
3046779 Jared Goff
3115252 Will Grier
16140 Ryan Griffin
14875 Robert Griffin III
4373632 Jaren Hall
2565969 Taylor Heinicke
4038941 Justin Herbert
2468609 Taysom Hill
2577128 Kevin Hogan
4426875 Sam Howell
12477 Bri

In [11]:
with open('player_stats_dictionary.json', 'w') as f:
    json.dump(full_dict, f, ensure_ascii=False, indent=4)

# Specific CURRENT College Players

In [14]:
url = 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes/3046779/statistics'

r = requests.get(url)
d = r.json()

In [17]:
r_2 = requests.get(d['athlete']['$ref'])
d_2 = r_2.json()

In [23]:
r_3 = requests.get(d_2['collegeAthlete']['$ref'])
d_3 = r_3.json()

In [26]:
r_4 = requests.get(d_3['statisticslog']['$ref'])
d_4 = r_4.json()

In [78]:
for x in d_4['entries']:
    for i in x['statistics']:
        if i['type'] == 'total':
            print(i['statistics']['$ref'])

http://sports.core.api.espn.com/v2/sports/football/leagues/college-football/seasons/2015/types/3/athletes/547401/statistics/0?lang=en&region=us
http://sports.core.api.espn.com/v2/sports/football/leagues/college-football/seasons/2014/types/3/athletes/547401/statistics/0?lang=en&region=us
http://sports.core.api.espn.com/v2/sports/football/leagues/college-football/seasons/2013/types/3/athletes/547401/statistics/0?lang=en&region=us


In [35]:
r_5 = requests.get(d_4['entries'][0]['statistics'][0]['statistics']['$ref'])
d_5 = r_5.json()

In [62]:
for x in d_5['splits']['categories']:
    print(x['name'])

general
passing
rushing
receiving
punting
scoring


In [64]:
d_5['splits']['categories'][0]['stats']

[{'name': 'fumbles',
  'displayName': 'Fumbles',
  'shortDisplayName': 'F',
  'description': 'The number of times a player/team has fumbled the ball',
  'abbreviation': 'FUM',
  'value': 0.0,
  'displayValue': '0'},
 {'name': 'fumblesLost',
  'displayName': 'Fumbles Lost',
  'shortDisplayName': 'FL',
  'description': 'The number of times a fumble is recovered by the opposing team',
  'abbreviation': 'LST',
  'value': 0.0,
  'displayValue': '0'},
 {'name': 'fumblesTouchdowns',
  'displayName': 'Fumbles Touchdowns',
  'shortDisplayName': 'FTD',
  'description': 'The number of times a fumbles is recovered and returned for a touchdown.',
  'abbreviation': 'FTD',
  'value': 0.0,
  'displayValue': '0'},
 {'name': 'gamesPlayed',
  'displayName': 'Games Played',
  'shortDisplayName': 'GP',
  'description': 'Games Played',
  'abbreviation': 'GP',
  'value': 13.0,
  'displayValue': '13'},
 {'name': 'offensiveTwoPtReturns',
  'displayName': 'Two Point Returns',
  'shortDisplayName': '2PTR',
  'de

In [82]:
for x in d_5['splits']['categories'][1]['stats']:
    print(x['name'], x['value'])

avgGain 0.0
completionPct 64.46099853515625
completions 341.0
ESPNQBRating 0.0
interceptionPct 2.4570000171661377
interceptions 13.0
longPassing 80.0
netPassingYards 4719.0
netPassingYardsPerGame 0.0
netTotalYards 4711.0
netYardsPerGame 0.0
passingAttempts 529.0
passingBigPlays 0.0
passingFirstDowns 0.0
passingFumbles 0.0
passingFumblesLost 0.0
passingTouchdownPct 8.128999710083008
passingTouchdowns 43.0
passingYards 4719.0
passingYardsAfterCatch 0.0
passingYardsAtCatch 4719.0
passingYardsPerGame 0.0
QBRating 161.30760192871094
sacks 26.0
sackYardsLost -200.0
teamGamesPlayed 13.0
totalOffensivePlays 611.0
totalPointsPerGame 0.0
totalTouchdowns 43.0
totalYards 4711.0
totalYardsFromScrimmage 3.0
twoPointPassConvs 0.0
twoPtPass 1.0
twoPtPassAttempts 4.0
yardsFromScrimmagePerGame 0.0
yardsPerCompletion 13.83899974822998
yardsPerGame 0.0
yardsPerPassAttempt 8.920999526977539
netYardsPerPassAttempt 8.502702702702702
quarterbackRating 60.41699981689453
